## 形態素解析の準備

* 形態素解析器は一旦janomeで

In [1]:
!pip install --user janome

## janomeテスト

In [2]:
from janome.tokenizer import Tokenizer
t = Tokenizer()
tokens = t.tokenize("短編小説")
for token in tokens:
    print(token)

短編	名詞,一般,*,*,*,*,短編,タンペン,タンペン
小説	名詞,一般,*,*,*,*,小説,ショウセツ,ショーセツ


## データ読み込み

* 提供されてる日経BPのメールリストを試しに利用
* 利用するためにローカルでちょっと前処理加えてます。

```bash
nkf -g nikkei_mail_magazine.csv
nkf -w --overwrite nikkei_mail_magazine.csv
nkf -g nikkei_mail_magazine.csv
sed -i.bak "1s/^.*$/\"date\",\"from\",\"subject\",\"body\"/" nikkei_mail_magazine.csv
```

SJIS to UTF8とheaderの日本語が扱いにくいので英語に変換

In [3]:

from pyspark.sql import SparkSession

# @hidden_cell
# This function accesses a file in your Object Storage. The definition contains your credentials.
# You might want to remove those credentials before you share your notebook.
def set_hadoop_config_with_credentials_a2229a240bdb40ca889517c3e9fffe02(name):
    """This function sets the Hadoop configuration so it is possible to
    access data from Bluemix Object Storage V3 using Spark"""

    prefix = 'fs.swift.service.' + name
    hconf = sc._jsc.hadoopConfiguration()
    hconf.set(prefix + '.auth.url', 'https://identity.open.softlayer.com'+'/v3/auth/tokens')
    hconf.set(prefix + '.auth.endpoint.prefix', 'endpoints')
    hconf.set(prefix + '.tenant', 'a54bf6fa500643c8be55e65ffa23044e')
    hconf.set(prefix + '.username', '6112cea574dd4a9785108a27d74be76a')
    hconf.set(prefix + '.password', 'Ax6THHtPEit6z?4_')
    hconf.setInt(prefix + '.http.port', 8080)
    hconf.set(prefix + '.region', 'dallas')
    hconf.setBoolean(prefix + '.public', False)

# you can choose any name
name = 'keystone'
set_hadoop_config_with_credentials_a2229a240bdb40ca889517c3e9fffe02(name)

spark = SparkSession.builder.getOrCreate()

df_data_8 = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load('swift://JapanBigDataHandsOn.' + name + '/nikkei_mail_magazine.csv')
df_data_8.take(5)


KeyboardInterrupt: 

In [ ]:
df_data_8.printSchema()

In [ ]:

df_data_6 = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load('swift://JapanBigDataHandsOn.' + name + '/analytics_staff.csv')
df_data_6.take(5)


In [ ]:
df_data_6.printSchema()

In [ ]:

df_data_7 = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load('swift://JapanBigDataHandsOn.' + name + '/analytics_job.csv')
df_data_7.take(5)


In [ ]:
df_data_7.printSchema()

## 対象データについて

一旦件名(subject)のみを対象とする。
本文(body)を外した理由としては、
* 記号系(http://~)の前処理面倒
* DSXのメモリに乗るか不安・・・

In [9]:
##
subject_rdd = df_data_8.select("subject").rdd.cache()
body_rdd = df_data_8.select("body").rdd.cache()

staff_text_rdd = df_data_6.select("STAFF_TEXT").rdd.cache()
job_text_rdd = df_data_7.select("JOB_TEXT").rdd.cache()


In [ ]:
#サンプル表示
#subject_rdd.take(1)
#body_rdd.take(1)
#job_text_rdd.take(1)
#staff_text_rdd.take(1)

## 形態素解析と品詞の確認
* 日経BPの情報(Subject)の確認

In [15]:
subject_rdd.count() ##全体のINPUT件数

2049

In [18]:
##形態素解析された中間データの生成
tokenized_subject_rdd = subject_rdd.filter(lambda x: x.subject != None)\
.map(lambda x: [(token.part_of_speech,(token.surface) if (token.base_form == "*") else token.base_form ) for token in Tokenizer().tokenize(x.subject)]).cache()

In [19]:
tokenized_subject_rdd.flatMap(lambda x: [(token[0],token[1].lower()) for token in x])\
.count() ##ディフォルトの辞書で形態素解析可能なワード数

13275

In [27]:
tokenized_subject_rdd.flatMap(lambda x:[(token[0],token[1].lower()) for token in x if (token[0].split(",")[0].find("名詞") >= 0 and token[0].split(",")[1].find("一般") >= 0)])\
.count() ##利用可能と思われる一般名詞

3110

In [ ]:
##出力が多いので注意！！！！
tokenized_subject_rdd.flatMap(lambda x:[(token[0],token[1].lower()) for token in x if (token[0].split(",")[0].find("名詞") >= 0 and token[0].split(",")[1].find("一般") >= 0)])\
.collect() ##利用可能と思われる一般名詞


In [30]:
##表現系のみ抽出
subject_input = tokenized_subject_rdd.map(lambda x:[token[1].lower() for token in x if (token[0].split(",")[0].find("名詞") >= 0 and token[0].split(",")[1].find("一般") >= 0)])
##word2vecのインスタンス生成
from pyspark.mllib.feature import Word2Vec
w2v = Word2Vec()
##subject_inputからモデル生成
subject_model = w2v.setNumIterations(3).fit(subject_input)

In [34]:
##モデルから10件ほどワードの近似をとる(ex セキュリティー)
for word,cos in subject_model.findSynonyms("セキュリティー",10):
    print("word: {}, cos: {}".format(word,cos))

word: 革命, cos: 0.9580185554272082
word: サミット, cos: 0.9566541933327893
word: メーカー, cos: 0.9426169766540321
word: 戦略, cos: 0.9305059926942794
word: 工場, cos: 0.9274528630415462
word: 手法, cos: 0.9264431645681588
word: エンジニア, cos: 0.9169012429213501
word: 未来, cos: 0.9164756517407582
word: mail, cos: 0.9119419929851151
word: 講座, cos: 0.9091678869524277


In [ ]:
##このセルはエラーが表示されます！！！！
##揺らぎ・認識していないワードのエラー(ex セキュリティ)
for word,cos in subject_model.findSynonyms("セキュリティ",10):
    print("word: {}, cos: {}".format(word,cos))

## subjectの考察
* 10件の近似したワードの出力は全体的に高い類似度を取っている
    * これはInputが少ない or かなり偏りがあるため同一のsubjectのみだとワードが偏っていると思われる
* セキュリティーというワードに対して革命というワードの重要度が高いのは正しいのか？
    * 普通に考えて無駄なワード・・・
* ワードの揺らぎ（セキュリティー/セキュリティ）の扱い

* bodyの確認

In [37]:
##bodyの件数確認
body_rdd.count()

2049

In [40]:
##形態素解析された中間データの生成
tokenized_body_rdd = body_rdd.filter(lambda x: x.body != None)\
.map(lambda x: [(token.part_of_speech,(token.surface) if (token.base_form == "*") else token.base_form ) for token in Tokenizer().tokenize(x.body)]).cache()

In [41]:
tokenized_body_rdd.flatMap(lambda x: [(token[0],token[1].lower()) for token in x])\
.count() ##ディフォルトの辞書で形態素解析可能なワード数

997616

In [42]:
tokenized_body_rdd.flatMap(lambda x:[(token[0],token[1].lower()) for token in x if (token[0].split(",")[0].find("名詞") >= 0 and token[0].split(",")[1].find("一般") >= 0)])\
.count() ##利用可能と思われる一般名詞

194003

In [43]:
##表現系のみ抽出
body_input = tokenized_body_rdd.map(lambda x:[token[1].lower() for token in x if (token[0].split(",")[0].find("名詞") >= 0 and token[0].split(",")[1].find("一般") >= 0)])
##subject_inputからモデル生成
body_model = w2v.setNumIterations(3).fit(body_input)

In [44]:
##モデルから10件ほどワードの近似をとる(ex セキュリティー)
for word,cos in body_model.findSynonyms("セキュリティー",10):
    print("word: {}, cos: {}".format(word,cos))

word: 脆弱, cos: 0.724601132346835
word: コンテスト, cos: 0.6800820966066703
word: セキュリ, cos: 0.6687638073249337
word: ecu, cos: 0.660375655214382
word: volkswagen, cos: 0.6568711164156256
word: セキュリティーハッカソン, cos: 0.6466343062083438
word: linux, cos: 0.6368051432735068
word: ティー, cos: 0.6326350765407089
word: 官公庁, cos: 0.6229293605448595
word: 委員, cos: 0.6151273748508489


## bodyの考察
* subjectよりはマシな類似度が出てる
* linuxのワードとセキュリティーのワードはつながっていて問題ないと思われる
* 官公庁とセキュリティーも今回のケースでは意味を持ちそう

* 人材情報からの抽出

In [45]:
staff_text_rdd.count() ##全体のINPUT件数

46483

In [46]:
##形態素解析された中間データの生成
tokenized_staff_rdd = staff_text_rdd.filter(lambda x: x.STAFF_TEXT != None)\
.map(lambda x: [(token.part_of_speech,(token.surface) if (token.base_form == "*") else token.base_form ) for token in Tokenizer().tokenize(x.STAFF_TEXT)]).cache()

In [ ]:
tokenized_staff_rdd.flatMap(lambda x: [(token[0],token[1].lower()) for token in x])\
.count() ##ディフォルトの辞書で形態素解析可能なワード数

In [ ]:
tokenized_body_rdd.flatMap(lambda x:[(token[0],token[1].lower()) for token in x if (token[0].split(",")[0].find("名詞") >= 0 and token[0].split(",")[1].find("一般") >= 0)])\
.count() ##利用可能と思われる一般名詞

## 形態素解析と対象語句の抽出

* ここで一般名詞のみ抽出してるが、対象がこれで良いかは検証必要・・・
* 一般名詞のみだと語句の関連性が飛んでるきがする・・・
* 名詞のみだと記号系が邪魔なので前処理でどうにか・・・
* 今回のだとサ変活用とかも入れたほうが良いかも

In [18]:
subject_input = subject_rdd.filter(lambda x: x.subject != None)\
.map(lambda x: [(token.part_of_speech,(token.surface) if (token.base_form == "*") else token.base_form ) for token in Tokenizer().tokenize(x.subject)])\
.map(lambda x: [token[1].lower() for token in x if (token[0].find("名詞") >= 0 and token[0].find("一般") >= 0)] )

body_input = body_rdd.filter(lambda x: x.body != None)\
.map(lambda x: [(token.part_of_speech,(token.surface) if (token.base_form == "*") else token.base_form ) for token in Tokenizer().tokenize(x.body)])\
.map(lambda x: [token[1].lower() for token in x if (token[0].find("名詞") >= 0 and token[0].find("一般") >= 0)] )

staff_input = staff_text_rdd.filter(lambda x: x.STAFF_TEXT != None)\
.map(lambda x: [(token.part_of_speech,(token.surface) if (token.base_form == "*") else token.base_form ) for token in Tokenizer().tokenize(x.STAFF_TEXT)])\
.map(lambda x: [token[1].lower() for token in x if (token[0].find("名詞") >= 0 and token[0].find("一般") >= 0)] )

job_input = job_text_rdd.filter(lambda x: x.JOB_TEXT != None)\
.map(lambda x: [(token.part_of_speech,(token.surface) if (token.base_form == "*") else token.base_form ) for token in Tokenizer().tokenize(x.JOB_TEXT)])\
.map(lambda x: [token[1].lower() for token in x if (token[0].find("名詞") >= 0 and token[0].find("一般") >= 0)] )



In [21]:
input = subject_input.union(body_input).union(job_input).union(staff_input)

#job_input.take(5)
#staff_input.take(2)

## word2vecモデル構築

* とりあえずIteration:3で。データ量と時間が許すならもうちょいIterationあげたほうが精度良くなる気がしてる

In [22]:
from pyspark.mllib.feature import Word2Vec
w2v = Word2Vec()
model = w2v.setNumIterations(3).fit(input)

## Synonyms

* 単語から単純な類似度の抽出。
* データ量が(ry
* やはり前処理が・・・・
* エラー出る場合はその単語が存在してない可能性が微レ存

In [11]:
#単語のcosを取る
for word,cos in model.findSynonyms("セキュリティ",10):
    print("word: {}, cos: {}".format(word,cos))


word: サイバーセキュリティ, cos: 0.717802432021995
word: jaspar, cos: 0.7129278982548151
word: ota, cos: 0.686505864218765
word: 脅威, cos: 0.6472447891425471
word: ミドル, cos: 0.6055930169931124
word: dena, cos: 0.5891736442972841
word: os, cos: 0.5873115911159915
word: システムズ, cos: 0.5794159866840655
word: 主査, cos: 0.5651777844114697
word: インターネット, cos: 0.5633326928571022


## wrodclusterの生成

* word2vecの結果からwordのclusterを生成。
* これで単語毎のgroupingは可能ここでは一旦10cluster。
* 多分業種 or 単語treeのrootノード数とかで適当にcluster化すればいいと思うの。
* ここから有向グラフを生成する方法・・・・はどうしよう・・・(；´Д｀)
* ここも精度はご察し・・・

In [ ]:
from pyspark.mllib.clustering import KMeans, KMeansModel
w2v_vectors = model.getVectors() #dic
l_vec = [ [float(c) for c in v] for v in w2v_vectors.values()]
cluster_n = 10
kmean_model = KMeans.train(sc.parallelize(l_vec),cluster_n,5)

for key,val in w2v_vectors.items():
    print("key:{},cluster:{}".format(key,kmean_model.predict([float(conv) for conv in val])))

## とりあえずの課題
* clusterからrootノードの抽出
* 関連性の上下関係
* 多分ワードが多くなると今度は速度が問題になるので、ワードの刈り込み or 概念でのクラス化

<br/>あたりが課題か？

## 複合語の生成

* 単純な複合語の生成について考察
    * 人名の生成（姓+名）
    * 名詞形容詞複合（形容動詞語幹+一般）
    * 名詞サ変複合（一般+サ変接続+接尾）

In [12]:
combi_input = body_rdd.filter(lambda x: x.body != None)\
.map(lambda x: [(token.part_of_speech,(token.surface) if (token.base_form == "*") else token.base_form ) for token in Tokenizer().tokenize(x.body)])\
.map(lambda x: [(token[0],token[1].lower()) for token in x if (token[0].startswith("名詞")
                                                               or token[0].startswith("動詞")
                                                               or token[0].startswith("形容詞"))])\
.cache()


In [13]:
sample = combi_input.map(lambda x: [(i,word) for i,word in enumerate(x)]).take(1)

In [14]:
new_words = []
stack = ""
last = None
for index,val in sample[0]:
    if(index == 0 or index == 1):
        continue
    last = sample[0][index-1:index]
    l_last = sample[0][index-2:index-1]
    (p,surf) = val
    p1,p2,p3,p4 = p.split(",")
    [(pi,(p_p,p_surf))] = last
    pp1,pp2,pp3,pp4 = p_p.split(",")
    [(ppi,(pp_p,pp_surf))] = l_last
    ppp1,ppp2,ppp3,ppp4 = pp_p.split(",")
    n_word = p_surf + surf
    if((pp4 == "姓" and p4 == "名")
       or (pp2 == "形容動詞語幹" and p2 == "一般")):
        new_words.append((pi,2,n_word,p_p))
    if(ppp2 == "一般" and pp2 == "サ変接続" and p2 == "接尾"):
        new_words.append((ppi,3,pp_surf + n_word,pp_p))
        

print(new_words)

[(48, 2, '重要課題', '名詞,形容動詞語幹,*,*'), (169, 2, '重要課題', '名詞,形容動詞語幹,*,*'), (203, 2, '重要課題', '名詞,形容動詞語幹,*,*'), (223, 2, '危険クルマ', '名詞,形容動詞語幹,*,*'), (357, 3, 'コンテスト開催日', '名詞,一般,*,*'), (500, 3, '日系完成車', '名詞,一般,*,*'), (548, 2, '安全環境', '名詞,形容動詞語幹,*,*'), (552, 3, '講師選定中', '名詞,一般,*,*'), (606, 3, 'セミナー終了後', '名詞,一般,*,*'), (610, 3, 'ネットワーキングパーティー交流会', '名詞,一般,*,*'), (722, 3, '定期購読者', '名詞,一般,*,*'), (834, 3, '部品開発者', '名詞,一般,*,*'), (851, 3, '世界中研究者', '名詞,一般,*,*'), (879, 2, '井上博之', '名詞,固有名詞,人名,姓'), (893, 2, '倉地亮', '名詞,固有名詞,人名,姓'), (971, 3, '科学研究科', '名詞,一般,*,*'), (979, 2, '井上博之', '名詞,固有名詞,人名,姓'), (985, 3, '科学研究科', '名詞,一般,*,*'), (996, 2, '倉地亮', '名詞,固有名詞,人名,姓'), (1115, 3, '定期購読者', '名詞,一般,*,*')]


## 新たなwordでの置き換え

In [17]:
#del sample[0][48:50]
#sample[0].insert(48,new_words[0])
print(sample[0][47:51])

[(47, ('名詞,サ変接続,*,*', '開発')), (48, ('名詞,形容動詞語幹,*,*', '重要')), (49, ('名詞,一般,*,*', '課題')), (50, ('名詞,サ変接続,*,*', '┃'))]


In [18]:
del sample[0][48:50]
print(sample[0][47:51])

[(47, ('名詞,サ変接続,*,*', '開発')), (50, ('名詞,サ変接続,*,*', '┃')), (51, ('名詞,サ変接続,*,*', '┗━◎')), (52, ('名詞,固有名詞,組織,*', 'http'))]


In [19]:
sample[0].insert(48,new_words[0])
print(sample[0][47:51])

[(47, ('名詞,サ変接続,*,*', '開発')), (48, 2, '重要課題', '名詞,形容動詞語幹,*,*'), (50, ('名詞,サ変接続,*,*', '┃')), (51, ('名詞,サ変接続,*,*', '┗━◎'))]


In [29]:
!id

uid=37476(s321-f440ad61ef9e72-ef22f67700fb) gid=100(users) groups=100(users)
